### Exploring CpGs for heteroskedasticity and differential methylation with passage
This code it an illustrative example and the python script 02_heteroskedasicity_CpGs.py was run with arguments for which iteration of 1,000. 

In [1]:
import numpy as np
import statsmodels
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sys
import statistics 
import random

In [2]:
import os
os.chdir('../')
os.getcwd()

'/home/redgar/Documents/DNAm_organoid_passage'

In [3]:
beta = pd.read_csv('data/beta_organoids.csv')

In [10]:
meta = pd.read_csv('data/meta_organoids.csv')

In [11]:
# prepare passage column from linear modelling
meta.rename(columns={"passage.or.rescope.no": "passage", "sample.type": "sampletype"}, inplace=True)

#df['score_num'] = df['score'].apply(score_to_numeric)
meta['passage'] = meta['passage'].str.replace('P','')
meta['passage'] = meta['passage'].str.replace('RE1.','')
meta['passage'] = pd.to_numeric(meta['passage'])

In [12]:
print('Samples in total cohort: ',meta.shape[0])
meta.head(3)

Samples in total cohort:  80


,Unnamed: 0,case.no,array.id,age.group,diagnosis,sample.site,sampletype,sampling.time.point,passage,sex,age,sentrix_ID,sentrix.pos,duplicated,sample_ID,array.id.path,det_pval,passage.or.rescope.no_numeric
0,1,212,201172580052_R02C01,paediatric,NaN,SC,organoid,original,6,M,11,201172580052,R02C01,Y,212 SC,/nfs/research1/zerbino/redgar/DNAm_organoid_pa...,0.000215,6
1,2,212,201172580051_R03C01,paediatric,NaN,TI,organoid,original,6,M,11,201172580051,R03C01,Y,212 TI,/nfs/research1/zerbino/redgar/DNAm_organoid_pa...,0.000207,6
2,3,223,201172560046_R01C01,paediatric,NaN,SC,organoid,original,2,F,15,201172560046,R01C01,Y,223 SC,/nfs/research1/zerbino/redgar/DNAm_organoid_pa...,0.000405,2


In [13]:
meta['passage'].value_counts(sort=False)

1      7
2     35
3      9
4      7
6      5
7      7
8      3
11     2
14     4
16     1
Name: passage, dtype: int64

Here is an example of the sample cohort subsampling to balance the number of low pasage samples. This is done iteratively in the for loop below.

In [14]:
random.seed(1)

## Sample the cohort in the lower passage number samples. 
#Pull 5 random samples from each of those with 1,2,3 or 4 passages.

meta_sampled_high_passage = meta[meta['passage'] > 4]
meta_sampled = meta[meta['passage'] <= 4]
meta_sampled_grouped = meta_sampled.groupby('passage')

meta_sampled_subset = []
for name, group in meta_sampled_grouped:
    meta_sampled_subset.append(group.sample(5))

meta_sampled_subset = pd.concat([pd.concat(meta_sampled_subset),meta_sampled_high_passage])

meta_sampled_subset['passage'].value_counts(sort=False)

1     5
2     5
3     5
4     5
6     5
7     7
8     3
11    2
14    4
16    1
Name: passage, dtype: int64

This notebook is a minimal example with only 20 CpGs instead of the 800,383 CpGs in the full dataset. In 02_heteroskedasicity_CpGs.py script the 1000 subsamplings were fed in as arguments. 

This selected a random subsample of n=5 samples for organoids from passage 1-4, and all organoids > 4 passages. Then in the sub sample at each CpG it runs a breusch pagan test for heteroskedasticity and differential DNA methylation.

In [15]:
#permstart = int(sys.argv[1])
#permend = int(sys.argv[1])+10
#CpGnum = beta.shape[0]

# set for minimal example
permstart = 1
permend = 5
CpGnum = 20 

pval_all_BP = []
pval_all_diff = []
db_all_diff = []
fdr_all_BP = []
fdr_all_diff = []

for n in range(permstart,permend):
    
    random.seed(n)

    ## Sample the cohort in the lower passage number samples. 
    #Pull 5 random samples from each of those with 1,2,3 or 4 passages.

    meta_sampled_high_passage = meta[meta['passage'] > 4]

    meta_sampled = meta[meta['passage'] <= 4]

    meta_sampled_grouped = meta_sampled.groupby('passage')

    meta_sampled_subset = []
    for name, group in meta_sampled_grouped:
        meta_sampled_subset.append(group.sample(5))

    meta_sampled_subset = pd.concat([pd.concat(meta_sampled_subset),meta_sampled_high_passage])


    ## collect a p value for each CpG

    beta_sampled = beta[meta_sampled_subset['array.id'].values.tolist()]

    CpG_pval_passage_subset = []
    CpG_pval_BP_subset = []
    CpG_db_passage_subset = []


    for cpg in range(0, CpGnum): #beta_sampled.shape[0]

        meta_sampled_subset['beta'] = beta_sampled.iloc[cpg,0:42].values.tolist()
        meta_sampled_subset['constant'] = 1

        reg = smf.ols('beta ~ passage', data=meta_sampled_subset).fit()
        # Differential p value is interesting as well
        pval_passage = reg.pvalues[1]
        db = (reg.params[1]*1)-(reg.params[1]*16)

        pred_val = reg.fittedvalues.copy()
        true_val = meta_sampled_subset['beta'].values.copy()
        residual = true_val - pred_val

        #BP heteroskedacity test
        _, pval_BP, __, f_pval = statsmodels.stats.diagnostic.het_breuschpagan(residual, meta_sampled_subset[['passage','constant']])
        # studentized or not (p vs f) values do match the ones from bptest in R

        CpG_pval_BP_subset.append(pval_BP)
        CpG_pval_passage_subset.append(pval_passage)
        CpG_db_passage_subset.append(db)
        
    pval_all_BP.append(CpG_pval_BP_subset)
    pval_all_diff.append(CpG_pval_passage_subset)
    db_all_diff.append(CpG_db_passage_subset)
    fdr_all_BP.append(statsmodels.stats.multitest.multipletests(CpG_pval_BP_subset, method='fdr_bh', is_sorted=False, returnsorted=False)[1])
    fdr_all_diff.append(statsmodels.stats.multitest.multipletests(CpG_pval_passage_subset, method='fdr_bh', is_sorted=False, returnsorted=False)[1])

In [16]:
pval_BP_df = pd.DataFrame(pval_all_BP)
pval_diff_df = pd.DataFrame(pval_all_diff)
db_all_diff = pd.DataFrame(db_all_diff)
fdr_all_BP = pd.DataFrame(fdr_all_BP)
fdr_all_diff = pd.DataFrame(fdr_all_diff)

In this example there were 4 iterations on 20 CpGs

In [17]:
print(pval_diff_df.shape)
print(fdr_all_diff)

(4, 20)
          0         1         2         3         4         5         6  \
0  0.958997  0.958997  0.005955  0.958997  0.958997  0.958997  0.480186   
1  0.424953  0.534542  0.005505  0.705561  0.903320  0.488006  0.184622   
2  0.425493  0.849937  0.034395  0.758626  0.972749  0.425493  0.189928   
3  0.508536  0.531528  0.128748  0.847637  0.847637  0.508536  0.460929   

          7         8         9        10        11        12        13  \
0  0.958997  0.958997  0.958997  0.480186  0.958997  0.958997  0.958997   
1  0.813948  0.701095  0.979843  0.424953  0.775570  0.775570  0.767480   
2  0.849937  0.425493  0.972749  0.189928  0.758177  0.972749  0.972749   
3  0.847637  0.847637  0.956245  0.508536  0.847637  0.847637  0.956245   

         14        15        16        17        18        19  
0  0.958997  0.958997  0.958997  0.958997  0.958997  0.694207  
1  0.703340  0.775570  0.813948  0.424953  0.892696  0.534542  
2  0.425493  0.849937  0.972749  0.644592  0.849

In [12]:
sig_BP = []
for cpg in range(0, CpGnum): #beta_sampled.shape[0]
    sig = sum(pval_BP_df.iloc[:,cpg] < 0.05)
    sig_BP.append(sig)
    
sig_diff = []
for cpg in range(0, CpGnum): #beta_sampled.shape[0]
    sig = sum(pval_diff_df.iloc[:,cpg] < 0.05)
    sig_diff.append(sig)
    
mn_db = []
for cpg in range(0, CpGnum): #beta_sampled.shape[0]
    mn = statistics.mean(db_all_diff.iloc[:,cpg])
    mn_db.append(mn)
    
sig_BP_fdr = []
for cpg in range(0, CpGnum): #beta_sampled.shape[0]
    sig = sum(fdr_all_BP.iloc[:,cpg] < 0.05)
    sig_BP_fdr.append(sig)

sig_diff_fdr = []
for cpg in range(0, CpGnum): #beta_sampled.shape[0]
    sig = sum(fdr_all_diff.iloc[:,cpg] < 0.05)
    sig_diff_fdr.append(sig)

Exporting the differenital methylation and heteroskedasticity (BP) p value, FDR p values and delta beta with passage for each of the 20 CpGs.  

In [13]:
pval_BP_df = pd.DataFrame([sig_BP, sig_diff,mn_db, sig_BP_fdr,sig_diff_fdr])
pval_BP_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.00000,4.000000,0.000000,0.000000,4.000000,4.000000
1,0.000000,0.000000,4.00000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.004252,0.001615,0.04779,-0.023742,0.001282,0.016993,-0.047391,0.005637,-0.001677,-0.003159,0.028824,0.006383,-0.014573,-0.001535,0.00221,0.023227,-0.011019,0.002313,0.025207,0.014691
3,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [41]:
#pval_BP_df.to_csv("data/passage_CpG_iterations/Heteroskedactiy_pvalues" + sys.argv[2] + ".csv","w")
pval_BP_df.to_csv("data/passage_CpG_iterations/Heteroskedactiy_pvalues_example.csv")